<a href="https://colab.research.google.com/github/JoseManuel009/03MAIR-Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2%2Cno_borrar)_Jos%C3%A9_Manuel_Ballester_Marhuenda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: JOSÉ MANUEL BALLESTER MARHUENDA  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/19Nw1eieypR6vWv5OGjLl_UIthswnWWlh#scrollTo=hVbXYX-RfPWh <br>
Problema:

>2. Organizar los horarios de partidos de una jornada de La Liga<br>


Descripción del problema:Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.







                                        

#Modelo

**¿Como represento el espacio de soluciones?**

El espacio de soluciones está representado por las asignaciones de los horarios a los partidos.

Cada solución será una lista de 10 horarios (1 por partido)

En total las posibles soluciones serán 10^10 lo que hace inviable la búsqueda exhaustiva.

**¿Cual es la función objetivo?**

La función objetivo tal y como se nos propone en el enunciado será maximizar la audiencia esperada y se calcula como:

$$
\text{audiencia total}=\sum_{i=1}^{N} (\text{audiencia base del partido}_i ×  \text{coeficiente del horario}_i)
$$

Además, se aplicarán penalizaciones si hay varios partidos en un mismo horario

**¿Como implemento las restricciones?**

Tal y como se nos dice en el enunciado se fuerza a que haya al menos un partido el viernes a las 20:00 y otro el lunes a las 20:00, esto se especificará en una función denominada validacion_restricciones().

Además también se penalizará la asignación de múñtiples partidos en el mismo horario, con una funcion llamada penalizaciones(cont), reduciendo la audiencia total cuando hay coincidencias.

A continuación, se muestran ambas funciones:


In [17]:
#Restricción ya que se sabe que al menos tiene que haber un partido el viernes y otro el lunes
def validacion_restricciones(asignacion):
  return 'Viernes_20' in asignacion and 'Lunes_20' in asignacion

#Penalización coincidencias
def penalizacion(cont):
  penalizacion=[0,0.25,0.45,0.60,0.70,0.75,0.78,0.80,0.80]
  return penalizacion[min(cont,8)]

#Análisis

**¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

Como se puede observar hay 10^10 posibles asignaciones, la evaluacuación de cada solución tiene una complejidad de O(n), donde n es el número de partidos, en este caso 10.

Con búsqueda exhaustiva el orden de complejida sería O(10^10)

Sin embargo, como búsqueda exhaustiva es inviable, yo voy a resolver el problema con un algorítmo genético, cuya complejidad si nos basamos en la teoría viene dada como O(T x P x n), donde T es el número de generaciones que he fijado en 200, P es el tamaño de población que en este caso es 100 y n es el número de partidos que ya se ha visto que son 10.

De tal forma que, la complejidad final usando algorítmos genéticos será O(200 x 100 x 10)=O(2 x 10^5), que es mucho más manejable.

#Diseño


**¿Que técnica utilizo? ¿Por qué?**

Como he indicado en el apartado anterior voy a usar un algorítmo genético para resolver el problema, porque por un lado el espacio de búqueda es muy grande para la optimización exhaustiva, por otro lado, el problema no tiene una estructura matemática clara que permita resolverlo con programación lineal y por último hay que tener en cuenta que los algorítmos genéticos encuentran muy buenas soluciones rápidamente y son relativamente fáciles de implementar.

Los algorítmos genéticos encuentran buenas soluciones mediante:

-Selección(para elegir las mejores soluciones)

-Cruce(Combinación de las soluciones para explorar nuevas)

-Mutación(introdución de diversidad para evitar estancamiento)

A continuación se va a mostrar la implementación del algorítmo genético para la tabla de partidos que se muestra en el enunciado y también he realizado la implementación para los partidos que se juegan este fin de semana:

## Algorítmo Genético para la asignación de partidos teniendo en cuenta los partidos del ejemplo

In [20]:
#Respuesta
import time
import random
import numpy as np
horarios_disponibles=['Viernes_20','Sábado_12','Sábado_16','Sábado_18','Sábado_20','Domingo_12','Domingo_16','Domingo_18','Domingo_20','Lunes_20']
coef_horario={'Viernes_20':0.4,'Sábado_12':0.55,'Sábado_16':0.7,'Sábado_18':0.8,'Sábado_20':1,'Domingo_12':0.45,'Domingo_16':0.75,'Domingo_18':0.8,'Domingo_20':1,'Lunes_20':0.4}

#Restricción ya que se sabe que al menos tiene que haber un partido el viernes y otro el lunes
def validacion_restricciones(asignacion):
  return 'Viernes_20' in asignacion and 'Lunes_20' in asignacion

#Penalización coincidencias
def penalizacion(cont):
  penalizacion=[0,0.25,0.45,0.60,0.70,0.75,0.78,0.80,0.80]
  return penalizacion[min(cont,8)]

#Datos de audiencia dependiendo del tipo de partido
audiencia_base={
    ('A','A'):2.0, ('A','B'):1.3, ('A','C'):1.0,
    ('B','A'):1.3,('B','B'):0.9,('B','C'):0.75,('C','C'):0.47
}
#Partidos ejemplo
partidos_ejemplo=[
    ('Celta','Real Madrid','B','A'),
    ('Valencia','Real Sociedad','B','A'),
    ('Mallorca','Eibar','C','C'),
    ('Athletic','Barcelona','B','A'),
    ('Leganés','Osasuna','C','C'),
    ('Villarreal','Granada','B','C'),
    ('Alavés','Levante','B','B'),
    ('Espanyol','Sevilla','B','B'),
    ('Betis','Valladolid','B','B'),
    ('Atlético','Getafe','B','B')
]
#Función de evaluación
def calculo_audiencia(asignacion):
  if not validacion_restricciones(asignacion):
    return 0
  contar_horarios={h:0 for h in horarios_disponibles}
  audiencia_total=0
  for i,horario in enumerate(asignacion):
    equipo1,equipo2,categoria1,categoria2=partidos_ejemplo[i]
    audiencia_base_partido=audiencia_base[(categoria1,categoria2)]
    coef=coef_horario[horario]
    contar_horarios[horario]+=1
    audiencia_total+=audiencia_base_partido*coef

  #Aplicacion de la penalización por coincidencias
  for h in horarios_disponibles:
    if contar_horarios[h]>1:
      penal=penalizacion(contar_horarios[h]-1)
      audiencia_total*=(1-penal)
  return audiencia_total

#Algorítmo genético
TAMAÑO_POP=100
GENERACIONES=200
RATIO_MUTACION=0.1

#Genero una población inicial
def generar_poblacion():
  poblacion=[]
  while len(poblacion)<TAMAÑO_POP:
    individuo=[random.choice(horarios_disponibles) for _ in partidos_ejemplo]
    if validacion_restricciones(individuo):
      poblacion.append(individuo)
  return poblacion

#Selección por torneo
def seleccion(poblacion):
  torneo=random.sample(poblacion,5)
  return max(torneo, key=calculo_audiencia)

#Cruce de los padres
def cruce(padre1,padre2):
  punto=random.randint(1,len(partidos_ejemplo)-1)
  hijo1=padre1[:punto]+padre2[punto:]
  hijo2=padre2[:punto]+padre1[punto:]
  return hijo1,hijo2

#Mutación
def mutacion(individuo):
  if random.random()<RATIO_MUTACION:
    i=random.randint(0,len(partidos_ejemplo)-1)
    individuo[i]=random.choice(horarios_disponibles)
  return individuo

#Algorítmo genético principal
def algoritmo_genetico():
  poblacion=generar_poblacion()
  for _ in range(GENERACIONES):
    nueva_poblacion=[]
    for _ in range(TAMAÑO_POP // 2):
      padre1=seleccion(poblacion)
      padre2=seleccion(poblacion)
      hijo1,hijo2=cruce(padre1,padre2)
      nueva_poblacion.extend([mutacion(hijo1),mutacion(hijo2)])
    poblacion=nueva_poblacion

  mejor_solucion=max(poblacion,key=calculo_audiencia)
  return mejor_solucion, calculo_audiencia(mejor_solucion)
inicio=time.time()
#Ejecuto el algorítmo
mejor_asignacion, mejor_audiencia=algoritmo_genetico()
fin=time.time()
print(f'Tiempo de ejecución: {fin-inicio} segundos')
print('Mejor asignación de horarios:')
for i,horario in enumerate(mejor_asignacion):
  print(f'{partidos_ejemplo[i][0]} vs {partidos_ejemplo[i][1]} -> {horario}')
print(f'Audiencia total esperada:{mejor_audiencia:.2f} millones')

Tiempo de ejecución: 0.9312498569488525 segundos
Mejor asignación de horarios:
Celta vs Real Madrid -> Domingo_20
Valencia vs Real Sociedad -> Domingo_18
Mallorca vs Eibar -> Lunes_20
Athletic vs Barcelona -> Domingo_12
Leganés vs Osasuna -> Viernes_20
Villarreal vs Granada -> Sábado_20
Alavés vs Levante -> Sábado_18
Espanyol vs Sevilla -> Sábado_12
Betis vs Valladolid -> Sábado_16
Atlético vs Getafe -> Domingo_16
Audiencia total esperada:6.57 millones


## Algorítmo Genético para la asignación de partidos teniendo en cuenta los partidos de la jornada actual

In [21]:
#Respuesta
import time
import random
import numpy as np
horarios_disponibles=['Viernes_20','Sábado_12','Sábado_16','Sábado_18','Sábado_20','Domingo_12','Domingo_16','Domingo_18','Domingo_20','Lunes_20']
coef_horario={'Viernes_20':0.4,'Sábado_12':0.55,'Sábado_16':0.7,'Sábado_18':0.8,'Sábado_20':1,'Domingo_12':0.45,'Domingo_16':0.75,'Domingo_18':0.8,'Domingo_20':1,'Lunes_20':0.4}

#Restricción ya que se sabe que al menos tiene que haber un partido el viernes y otro el lunes
def validacion_restricciones(asignacion):
  return 'Viernes_20' in asignacion and 'Lunes_20' in asignacion

#Penalización coincidencias
def penalizacion(cont):
  penalizacion=[0,0.25,0.45,0.60,0.70,0.75,0.78,0.80,0.80]
  return penalizacion[min(cont,8)]

#Datos de audiencia dependiendo del tipo de partido
audiencia_base={
    ('A','A'):2.0, ('A','B'):1.3, ('A','C'):1.0,
    ('B','A'):1.3,('B','B'):0.9,('B','C'):0.75,('C','B'):0.75,('C','C'):0.47
}
#Partidos jornada 26 de la liga 2024/2025
partidos_jornada_actual=[
    ('Valladolid','Las Palmas','C','C'),
    ('Girona','Celta','B','B'),
    ('Rayo Vallecano','Sevilla','B','B'),
    ('Betis','Real Madrid','B','A'),
    ('Atlético','Athletic','A','B'),
    ('Leganés','Getafe','C','B'),
    ('Barcelona','Real Sociedad','A','B'),
    ('Mallorca','Alavés','B','B'),
    ('Osasuna','Valencia','B','B'),
    ('Villarreal','Espanyol','B','B')
]
#Función de evaluación
def calculo_audiencia(asignacion):
  if not validacion_restricciones(asignacion):
    return 0
  contar_horarios={h:0 for h in horarios_disponibles}
  audiencia_total=0
  for i,horario in enumerate(asignacion):
    equipo1,equipo2,categoria1,categoria2=partidos_jornada_actual[i]
    audiencia_base_partido=audiencia_base[(categoria1,categoria2)]
    coef=coef_horario[horario]
    contar_horarios[horario]+=1
    audiencia_total+=audiencia_base_partido*coef

  #Aplicacion de la penalización por coincidencias
  for h in horarios_disponibles:
    if contar_horarios[h]>1:
      penal=penalizacion(contar_horarios[h]-1)
      audiencia_total*=(1-penal)
  return audiencia_total

#Algorítmo genético
TAMAÑO_POP=100
GENERACIONES=200
RATIO_MUTACION=0.1

#Genero una población inicial
def generar_poblacion():
  poblacion=[]
  while len(poblacion)<TAMAÑO_POP:
    individuo=[random.choice(horarios_disponibles) for _ in partidos_jornada_actual]
    if validacion_restricciones(individuo):
      poblacion.append(individuo)
  return poblacion

#Selección por torneo
def seleccion(poblacion):
  torneo=random.sample(poblacion,5)
  return max(torneo, key=calculo_audiencia)

#Cruce de los padres
def cruce(padre1,padre2):
  punto=random.randint(1,len(partidos_jornada_actual)-1)
  hijo1=padre1[:punto]+padre2[punto:]
  hijo2=padre2[:punto]+padre1[punto:]
  return hijo1,hijo2

#Mutación
def mutacion(individuo):
  if random.random()<RATIO_MUTACION:
    i=random.randint(0,len(partidos_jornada_actual)-1)
    individuo[i]=random.choice(horarios_disponibles)
  return individuo

#Algorítmo genético principal
def algoritmo_genetico():
  poblacion=generar_poblacion()
  for _ in range(GENERACIONES):
    nueva_poblacion=[]
    for _ in range(TAMAÑO_POP // 2):
      padre1=seleccion(poblacion)
      padre2=seleccion(poblacion)
      hijo1,hijo2=cruce(padre1,padre2)
      nueva_poblacion.extend([mutacion(hijo1),mutacion(hijo2)])
    poblacion=nueva_poblacion

  mejor_solucion=max(poblacion,key=calculo_audiencia)
  return mejor_solucion, calculo_audiencia(mejor_solucion)
inicio=time.time()
#Ejecuto el algorítmo
mejor_asignacion, mejor_audiencia=algoritmo_genetico()
fin=time.time()
print(f'Tiempo de ejecución: {fin-inicio} segundos')
print('Mejor asignación de horarios:')
for i,horario in enumerate(mejor_asignacion):
  print(f'{partidos_jornada_actual[i][0]} vs {partidos_jornada_actual[i][1]} -> {horario}')
print(f'Audiencia total esperada:{mejor_audiencia:.2f} millones')

Tiempo de ejecución: 0.5503056049346924 segundos
Mejor asignación de horarios:
Valladolid vs Las Palmas -> Viernes_20
Girona vs Celta -> Domingo_18
Rayo Vallecano vs Sevilla -> Sábado_16
Betis vs Real Madrid -> Sábado_18
Atlético vs Athletic -> Sábado_12
Leganés vs Getafe -> Domingo_12
Barcelona vs Real Sociedad -> Domingo_20
Mallorca vs Alavés -> Domingo_16
Osasuna vs Valencia -> Lunes_20
Villarreal vs Espanyol -> Sábado_20
Audiencia total esperada:6.87 millones


# Conclusión

En conclusión, el algorítmo genético ofrece un buen equilibrio entre exploración y explotación, permitiendo encontrar soluciones bastante cercanas al óptimo en un tiempo más que razonable.